<a href="https://colab.research.google.com/github/orlandoalexander/Moda-Personal-Assistant/blob/models/Landmark_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from preproc.preprocess import LandmarksPreproc
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Conv2D,MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.applications import MobileNetV2,mobilenet
from sklearn.model_selection import train_test_split

In [1]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.8 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Cloning into 'Moda-Personal-Assistant'...
remote: Enumerating objects: 806, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 806 (delta 30), reused 56 (delta 19), pack-reused 725
Receiving objects: 100% (806/806), 248.12 MiB | 26.36 MiB/s, done.
Resolving deltas: 100% (421/421), done.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/orlandoalexander/Moda-Personal-Assistant.git@preproc_package


In [ ]:
!unzip '/content/drive/MyDrive/Le Wagon/Moda-Personal-Assistant/Data/img_sections:cats:landmarks.zip'

In [2]:
path = 'images'

In [3]:
prep = LandmarksPreproc(path, (256,256),0.2)

                                                    img     x1     y1     x2  \
0             MEN-Denim-id_00000080-01_7_additional.jpg  267.0  234.0  509.0   
1             MEN-Denim-id_00000089-01_7_additional.jpg  347.0  201.0  598.0   
2             MEN-Denim-id_00000089-02_7_additional.jpg  236.0  234.0  491.0   
3             MEN-Denim-id_00000089-03_7_additional.jpg  225.0  237.0  467.0   
4             MEN-Denim-id_00000089-04_7_additional.jpg  244.0  235.0  480.0   
...                                                 ...    ...    ...    ...   
28832  WOMEN-Tees_Tanks-id_00007969-04_7_additional.jpg  221.0  218.0  392.0   
28835  WOMEN-Tees_Tanks-id_00007970-01_7_additional.jpg  242.0  229.0  446.0   
28838        WOMEN-Tees_Tanks-id_00007976-01_4_full.jpg  288.0  225.0  473.0   
28841        WOMEN-Tees_Tanks-id_00007979-03_4_full.jpg  287.0  210.0  466.0   
28845        WOMEN-Tees_Tanks-id_00007979-04_4_full.jpg  275.0  187.0  468.0   

          y2     x3     y3     x4     y

In [4]:
X_train, X_test, y_train, y_test = prep.run()

Done!


In [5]:
es= EarlyStopping(patience=5, restore_best_weights=True)

input = Input(X_train.shape[1:])

# base_model = MobileNetV2(include_top=False, weights='imagenet',
#                                      classes=12, input_tensor=input)
# base_model.trainable = False
# pool = GlobalAveragePooling2D()
# dense = Dense(50, activation='relu')
# prediction = Dense(12, activation="linear")
# model = Sequential([base_model, pool, dense, prediction])
# preprocess_input = mobilenet.preprocess_input

model = Sequential()
model.add(Conv2D(64, (2, 2), input_shape=X_train.shape[1:], padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (4, 4), padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(12, activation="linear"))
model.compile(loss='mae', metrics=['mae'], optimizer='adam')

In [6]:
history = model.fit(X_train, y_train, validation_split = 0.2, epochs=100, callbacks = [es])

Epoch 1/100
183/183 [==============================] - 18s 71ms/step - loss: 50.4443 - mae: 50.4443 - val_loss: 33.9101 - val_mae: 33.9101
Epoch 2/100
183/183 [==============================] - 12s 63ms/step - loss: 35.1002 - mae: 35.1002 - val_loss: 30.6014 - val_mae: 30.6014
Epoch 3/100
183/183 [==============================] - 12s 64ms/step - loss: 31.4994 - mae: 31.4994 - val_loss: 30.2106 - val_mae: 30.2106
Epoch 4/100
183/183 [==============================] - 12s 64ms/step - loss: 29.9994 - mae: 29.9994 - val_loss: 27.9568 - val_mae: 27.9568
Epoch 5/100
183/183 [==============================] - 12s 64ms/step - loss: 27.1542 - mae: 27.1542 - val_loss: 32.2998 - val_mae: 32.2998
Epoch 6/100
183/183 [==============================] - 12s 64ms/step - loss: 26.5754 - mae: 26.5754 - val_loss: 34.3314 - val_mae: 34.3314
Epoch 7/100
183/183 [==============================] - 12s 64ms/step - loss: 26.4469 - mae: 26.4469 - val_loss: 26.2515 - val_mae: 26.2515
Epoch 8/100
183/183 [======

In [7]:
model.evaluate(X_test, y_test)

58/58 [==============================] - 1s 18ms/step - loss: 23.5646 - mae: 23.5646


[23.56456184387207, 23.56456184387207]